<a id='rdw_top'>

# Add extra information from RDW open data

Query to the open data dataset of the RDW.


1. <a href="#rdw_registrations">Registration numbers</a>  
    Apis with registration as key
2. <a href="#rdw_confcodes">Conformity codes</a>  
    Cars get a conformity code when certified.
3. <a href="#rdw_other_apis">Other APIs</a>  
    It may take a while (10 min) to query all conformity codes individually.
4. <a href="#rdw_ovi">Closed data</a>  
    Get closed data from RDW website. This also takes a while because of time out enforced by website. Use config to disable.
5. <a href="#rdw_merge">Merge results</a>  
    Combine all dataframes and save

- - - - 

### User variables


In [ ]:
import drz_config
cfg = drz_config.read_config()
DATE = cfg['DATE']
VERBOSE = cfg['VERBOSE']
OPBOD = cfg['OPBOD']
CLOSEDDATA = cfg['CLOSEDDATA']
closed_data_fields = cfg['closed_data_fields']
SKIPSAVE = cfg['SKIPSAVE']
TAG_RM_ALL = "nbconvert_instruction:remove_all_outputs" # tag to remove cell output to reduce "diff" output when comitting

QUICK_MERGE = False # check if rdw data already exist (ran when auction was still open)


if VERBOSE > 0:
    display(cfg)

### Modules and functions

In [2]:
import pandas as pd
import numpy as np
import re 
import os
# to keep api key hidden import this from sub dir
import assets.hidden_api_keys as hidden_api_keys
from time import sleep
import urllib

# base url
apiurl = 'https://opendata.rdw.nl/resource/m9d7-ebf2.json?$$app_token=' + hidden_api_keys.socrata_apptoken + '&'

def get_json_from_api(url,reg,c=0):
    
    '''Get json object from api'''
    
    import time

    c+=1
    try:
        df=pd.read_json(url + 'kenteken=' + reg.replace('-','').upper()).to_dict()
    except:
        if c > 10:
            print(url,reg)
            raise 
        else:
            print('pause 2 sec and try again!')
            time.sleep(2)
            df = get_json_from_api(url,reg,c)
    
    return df
    
# get_json_from_api(apiurl,'61-sf-FG')

### Load auction results

In [ ]:
if OPBOD:
    file_name = '../../../python-nb/data/drz-data-opbod-{}.pkl'.format(DATE)
else:
    file_name = '../data/drz-data-{}.pkl'.format(DATE)

if not os.path.isfile(file_name):
    # see if -without price- exists
    NO_PRICE = True
    file_name = file_name.replace('.pkl', '-without-price.pkl')
else:
    NO_PRICE = False

print(file_name)
drz = pd.read_pickle(file_name)

In [4]:
# load existing RDW data and merge with price
file_name = f'../data/rdw-data-{DATE}.pkl'
file_name = file_name.replace('.pkl', '-without-price.pkl')
if os.path.isfile(file_name) & QUICK_MERGE:
    print(file_name)
    rdw = pd.read_pickle(file_name)
    
    # add price
    rdw.update(drz.Price)

    # save
    out = rdw.copy()
    file_name = file_name.replace('-without-price.pkl','.pkl')
    if (SKIPSAVE==False) and (not(os.path.isfile(file_name))):
        print(file_name)
        out.to_pickle(file_name)
    else:
        print(f'Skip. {file_name} exists or saving is disabled in settings.')
        
    assert False, "Stop running. Everything is done"

### Collect registrations

In [ ]:
# see what lots have a Dutch registration (license number).
hasReg = (~drz.Reg.isnull()) & (drz.Reg != 'onbekend') & (drz.Reg != '') & (~drz.LotType.isin([
    'Vaartuig',
    'Jetski',
    'Sloep',
    'Speedboot',
    'Vaartuig (Type onbekend)',
    'Motorvaartuig met opbouw (Pleziervaartuig)',
]))

print('nr. of registrations:',sum(hasReg))

# # make a copy and add info
# rdw = drz.copy()


In [6]:
def get_query_url(api_url, keys, token=hidden_api_keys.socrata_apptoken, query_field='kenteken'):
    '''construct query url'''

    import urllib

    # First part
    q = api_url + '?$$app_token=' + token + '&'

    # convert list to string
    id_list = ''.join(["'{}', ".format(k) for k in keys])
    id_list = id_list[0:-2] # remove trailing ', '
    
    # add escaped soql
    soql = '$where=' + urllib.parse.quote(query_field + ' in (' + id_list + ')')
    q += soql
    
    # See if field is available
    accepted_fields = eval(urllib.request.urlopen(api_url).headers.get('X-SODA2-Fields'))
    if query_field not in accepted_fields:
        raise ValueError(f'<{query_field}> not allowed as SODA2 field in {api_url}.\n\tAccepted are: ' + ', '.join(accepted_fields))
    
    return q

Create list of dataframes with different api results

In [ ]:
# empty dictionary
rdw_per_reg = dict()

# first element of dict is registrations
key = 'registations'
rdw_per_reg[key] = drz.loc[hasReg,['Reg', 'LotType']].copy() # copy from drz
rdw_per_reg[key]['kenteken'] = rdw_per_reg[key].Reg.apply(lambda r: r.replace('-','').upper())
rdw_per_reg[key].index.name = 'lot_index'
rdw_per_reg[key] = rdw_per_reg[key].reset_index().set_index('kenteken')
display(rdw_per_reg[key].reset_index().set_index(['LotType', 'kenteken']))


<H1><a href="#rdw_top">^</a></H1><a id='rdw_registrations'>

# Main apis

The main api: `api_gekentekende_voertuigen` points to subsequent apis.

In [ ]:
# Assess these registrations
regs = rdw_per_reg['registations'].Reg.values
regs = [r.replace('-','').upper() for r in regs]
print(len(regs),'registrations in this set')

# Do main api first to get other possible apis
api_name = 'api_gekentekende_voertuigen'
api_url = 'https://opendata.rdw.nl/resource/m9d7-ebf2.json'

# Query api
key = re.sub('^api_','', api_name) # store with this key
q = get_query_url(api_url,regs)
rdw_per_reg[key] = pd.read_json(q)
rdw_per_reg[key].set_index('kenteken', inplace=True)

print(api_name, end=': ')
display(rdw_per_reg[key])

# Query other available apis
for api_name in [c for c in rdw_per_reg['gekentekende_voertuigen'].columns if c.startswith('api')]:
    print(api_name, end=': ')
    key = re.sub('^api_','',api_name)
    for api_url in rdw_per_reg['gekentekende_voertuigen'][api_name].unique():
        print(api_url)
        # query the web
        q = get_query_url(api_url,regs)
        df0 = pd.read_json(q)
        # name of index
        df0.columns.name = api_name

        # query should return 'kenteken', make it the index
        if df0.shape[0] != 0:
            df0.set_index('kenteken', inplace=True)
            
        # Some apis return multiple values. Pivot around index number ("volgnummer")
        if api_name == 'api_gekentekende_voertuigen_assen':
            df0 = pd.pivot(df0, columns='as_nummer')

        elif api_name == 'api_gekentekende_voertuigen_brandstof':
            df0 = pd.pivot(df0, columns='brandstof_volgnummer')

        elif api_name == 'api_gekentekende_voertuigen_carrosserie':
            df0 = pd.pivot(df0, columns='carrosserie_volgnummer')

        elif api_name == 'api_gekentekende_voertuigen_carrosserie_specifiek':
            df0 = pd.pivot(df0, columns='carrosserie_voertuig_nummer_code_volgnummer')

        # squeeze multi index
        one_level = [
            re.sub('^api_gekentekende_voertuigen_','',api_name) + '_' + '_'.join(
                [str(c) if type(c)==int else c for c in l]
            ) for l in df0.columns
        ]
        df0.columns = one_level

        # add to list
        display(df0.tail(3))
        print(df0.shape)
        rdw_per_reg[key]=df0

# Merge dataframes from different apis
df_regs = pd.concat(rdw_per_reg.values(), axis='columns', sort=False)
# add timestamp
df_regs['TimeStamp'] = pd.to_datetime('now').strftime('%Y%m%d')
# set lot id as index
df_regs.index.name = 'kenteken'
df_regs = df_regs.reset_index().set_index('lot_index')
display(df_regs)

<H1><a href="#rdw_top">^</a></H1><a id='rdw_confcodes'>

# Conformity codes

In [ ]:
# empty dict
rdw_per_confcode = dict()
# Conformity codes have sub-divisions. Four fields make a super key
key = 'conformity_codes'
rdw_per_confcode[key] = rdw_per_reg['gekentekende_voertuigen'][[
    'typegoedkeuringsnummer', 
    'uitvoering', 
    'variant', 
    'volgnummer_wijziging_eu_typegoedkeuring'
]].dropna().drop_duplicates()
rdw_per_confcode[key].reset_index(drop=True, inplace=True)
print(len(rdw_per_confcode[key]),'conformity codes in this set')

display(rdw_per_confcode[key])
print('count unique')
display(\
    rdw_per_reg['gekentekende_voertuigen'].loc[:, rdw_per_confcode[key].columns]\
    .reset_index()\
    .groupby([rdw_per_confcode[key].columns[0]])\
    .nunique()\
    .replace(1,np.NaN)\
    .dropna(how='all')\
    .fillna(1)\
    .astype(int)\
    .sort_values(by='kenteken', ascending=False)
)

In [ ]:
# do conformity api and again get other possible apis
api_name = 'api_eeg_voertuigtypegoedkeuring'
api_url = 'https://opendata.rdw.nl/resource/55kv-xf7m.json'

# Query api
key = re.sub('^api_','', api_name) # store with this key
q = get_query_url(
    api_url, 
    rdw_per_confcode['conformity_codes'].typegoedkeuringsnummer.unique(), # Use long (year with century) version of conformity code
    query_field='typegoedkeuringsnummer'
)
rdw_per_confcode[key] = pd.read_json(q)
# more than one conformity code?
assert not (rdw_per_confcode[key].groupby('typegoedkeuringsnummer')['typegoedkeuringsnummer'].count() > 1).any()
rdw_per_confcode[key].set_index('typegoedkeuringsnummer', inplace=True)

# add slightly different keys (year has no century)
rdw_per_confcode['conformity_codes'] = rdw_per_confcode['conformity_codes'].merge(
    rdw_per_confcode['eeg_voertuigtypegoedkeuring'].eu_type_goedkeuringssleutel, 
    how='left', 
    left_on='typegoedkeuringsnummer', 
    right_index=True
)

print(api_name, end=': ')
display(rdw_per_confcode[key])




<H1><a href="#rdw_top">^</a></H1><a id='rdw_other_apis'>

# Other apis

In [ ]:
# query other available apis
api_list = [c for c in rdw_per_confcode['eeg_voertuigtypegoedkeuring'].columns if c.startswith('api')]
for api_name in api_list:
    print(api_name, end=''.join(['|' if api_name==list_element else '.'  for list_element in api_list]))
    key = re.sub('^api_','',api_name)
    for api_url in rdw_per_confcode['eeg_voertuigtypegoedkeuring'][api_name].unique():
        
        # reformat url
        M=re.search('https://opendata.rdw.nl/.*/([a-z0-9]{4}-[a-z0-9]{4})$', api_url)
        api_url = 'https://opendata.rdw.nl/resource/{}.json'.format(M[1])
        print(api_url)
       
        # query the web
        # unfortunately this needs to be done one by one, because conformity code is not unique                   
        df0 = pd.DataFrame()
        for ix, row in rdw_per_confcode['conformity_codes'].iterrows():
            q = api_url
            q += '?$$app_token=' + hidden_api_keys.socrata_apptoken 
            q += '&{}=\'{}\''.format('eu_type_goedkeuringssleutel', urllib.parse.quote(row.eu_type_goedkeuringssleutel))
            q += '&{}=\'{}\''.format('eeg_uitvoeringscode', urllib.parse.quote(row.uitvoering))
            q += '&{}=\'{}\''.format('eeg_variantcode', urllib.parse.quote(row.variant))
            q += '&{}={:.0f}'.format('uitvoering_wijzigingsnummer', row.volgnummer_wijziging_eu_typegoedkeuring)
                
            n_try = 0
            OK = False
            while (n_try < 10) & (not OK):
                try:
                    res = pd.read_json(q)
                    OK = True
                    n_try = 0
                except:
                    n_try +=1
                    sleep(10)
                if n_try == 10:
                    raise

            if len(res) == 0:
                continue

            # matching data type with 'codes' for merging
            res.eu_type_goedkeuringssleutel = res.eu_type_goedkeuringssleutel.astype(str)
            res.eeg_uitvoeringscode = res.eeg_uitvoeringscode.astype(str)
            res.eeg_variantcode = res.eeg_variantcode.astype(str)
            res.uitvoering_wijzigingsnummer = res.uitvoering_wijzigingsnummer.astype(float)
            res.set_index([
                'eu_type_goedkeuringssleutel', 
                'eeg_uitvoeringscode', 
                'eeg_variantcode', 
                'uitvoering_wijzigingsnummer'
            ], inplace=True)
                
                
            # Api specific modifiers
            if api_name == 'api_as_gegevens_eeg_uitvoering':               
                piv = res.pivot(columns='asnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)                
            elif api_name == 'api_handelsbenaming_uitvoering':                
                piv = res.pivot(columns='volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_carrosserie_uitvoering':                
                piv = res.pivot(columns='carrosserie_volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_plaatsaanduiding_uitvoering':
                piv = res.pivot(columns='plaats_aanduiding_volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)            
            elif api_name == 'api_carrosserie_uitvoering_nummerieke_code':
                piv = res.pivot(columns='carrosserie_uitvoering_numeriek_volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)            
            elif api_name == 'api_uitvoeringverbruik_per_uitgave':
                piv = res.pivot(columns='uitvgavenummer_verbruikboek')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_motor_uitvoering':
                piv = res.pivot(columns='volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_versnellingsbak_uitvoering':
                piv = res.pivot(columns='volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_motor_uitvoering_brandstof':
                piv = pd.pivot_table(res, index=res.index.names, columns=['volgnummer', 'brandstof_volgnummer'])
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)                
            elif api_name == 'api_merk_uitvoering_toegestaan':
                piv = pd.pivot_table(res, index=res.index.names, values=res.columns, aggfunc=list)
                df0 = pd.concat([df0, piv], sort=False)
            else:
                assert res.shape[0] == 1 # Nothing to pivot on
                columns = ['{}_{}'.format(re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)), c) for c in res.columns] 
                res.columns = columns
                df0 = pd.concat([df0, res], sort=False)
                
            print('.', end='')
        if len(df0)==0:
            print('No results for this api')
            continue


        # add to dict
        rdw_per_confcode[key] = df0
        display(rdw_per_confcode[key].tail())
        print(rdw_per_confcode[key].shape)


<H1><a href="#rdw_top">^</a></H1><a id='rdw_ovi'>

# Closed data from rdw (OVI)
Optionally get data from rdw website

In [12]:
def get_closed_data(reg, fields = '*'):
    web_url = f'https://ovi.rdw.nl/?kenteken={reg}'

    import requests
    import codecs
    from lxml import html, etree
    import re
    from time import sleep
    
    def _all_fields(tree):
        elements = tree.xpath('//*[@class="ui-block-d border ovigrid"]') + tree.xpath('//*/div[@class="ui-block-d ovigrid"]')
        return [el.attrib['id'] for el in elements if 'id' in el.keys()]
    
    # get page
    page = requests.get(web_url)
    DecodeType = re.findall('charset=(.*)$', page.headers["Content-type"])[0]
    htmlstring = codecs.decode(page.content, DecodeType)
    # Convert string to tree object
    tree = html.fromstring(htmlstring)

    # analyze results
    ERROR = len(tree.xpath('//*[@action="FoutPagina.aspx?error=ErrorBlocked"]')) == 1
    warn_text = tree.xpath('//*[@class="warning"]/*/text()')
    WARNING_NA = (len(warn_text) > 0) and (warn_text[0].startswith('Er zijn geen gegevens gevonden voor het ingevulde kenteken '))

    # return result
    if ERROR:
        print('X', end='')
        return False
    elif WARNING_NA:
        if VERBOSE > 1: print(warn_text[0])
        print('x', end='')
        return None
    else:
        if fields == '*':
            fields = _all_fields(tree)
        out = dict()
        for fld in fields:
            txt = tree.xpath(f'//*[@id="{fld}"]/text()')
            if len(txt) != 1:
                # No result with this field. Just skip
                if VERBOSE > 1: print(fld)
                print('-', end='')
            else:
                print('.', end='')
                out[fld] = str(txt[0])
        # No result with these fields. Probably limit request
        if all([v is None for v in out.values()]):
            raise AssertionError(f'No result with fields: {", ".join(fields)}.')


    return out
    

In [ ]:
if CLOSEDDATA:
    data = {k: False for k in df_regs.kenteken}
    if VERBOSE > 1: print('X: error\n.: info retrieved\n-: N.A.')
    DO_LOOP = True
    c = 0 #counter
    max_loop = 2 * len(data) / 30 # approx nr of succesful retrievals is 30
    while DO_LOOP or c > max_loop:
        # pause
        if c > 0:
            if VERBOSE > 1: print(f' time out (60s) for next iteration', end='')
            sleep(60)
            if VERBOSE > 1: print(f': {c:2.0f}/{max_loop:2.0f}')
        c+=1 

        # has no value or field has no value
        to_do_regs = [k for k,v in data.items() if v == False]
        if VERBOSE > 0: print(f'\nto do:{len(to_do_regs):3.0f}', end=' ')
        data.update({reg: get_closed_data(reg, closed_data_fields) for reg in to_do_regs})
        DO_LOOP = any([v==False for v in data.values()])

    # drop None
    data = {k:v for k,v in data.items() if v is not None}    
    rdw_closed = pd.DataFrame.from_dict(data=data, orient='index')
else:
    rdw_closed = None

rdw_closed

<H1><a href="#rdw_top">^</a></H1><a id='rdw_merge'>

# Merge datasets
- Merge dataframes from conformity codes apis
- Merge with registration results
- Merge with auction results

In [ ]:
# first merge first two results
tmp = rdw_per_confcode['conformity_codes'].merge(rdw_per_confcode['eeg_voertuigtypegoedkeuring'], how='left', 
                                              left_on='typegoedkeuringsnummer',
                                              right_index=True
                                             )

tmp.rename(columns={
    'eu_type_goedkeuringssleutel_x': 'eu_type_goedkeuringssleutel',
    'uitvoering': 'eeg_uitvoeringscode',
    'variant': 'eeg_variantcode',
    'volgnummer_wijziging_eu_typegoedkeuring': 'uitvoering_wijzigingsnummer',
}, inplace=True)
tmp.set_index(['eu_type_goedkeuringssleutel', 'eeg_uitvoeringscode', 'eeg_variantcode', 'uitvoering_wijzigingsnummer'], inplace=True)

# merge with subsequent api results
df_confcodes = pd.concat([tmp] + list(rdw_per_confcode.values())[2:], axis='columns', sort=False)
# add timestamp
df_confcodes['TimeStamp'] = pd.to_datetime('now').strftime('%Y%m%d')

display(df_confcodes.tail())
print(df_confcodes.shape)

In [ ]:
# Merge confirmation codes with registrations
df_regs.index.name = 'lot_index'
df = df_regs.reset_index().merge(df_confcodes.reset_index(), how='left',
                   left_on=['typegoedkeuringsnummer', 'uitvoering', 'variant', 'volgnummer_wijziging_eu_typegoedkeuring'],
                   right_on=['typegoedkeuringsnummer', 'eeg_uitvoeringscode', 'eeg_variantcode', 'uitvoering_wijzigingsnummer'],
).set_index('lot_index')
display(df.tail(10))
print(df.shape)
assert all(df.columns.value_counts() == 1)

In [16]:
# Merge closed source data with registrations

# make fields lowercase and add "ovi_"
rdw_closed.index.name='kenteken'
rdw_closed.columns = ['ovi' + re.sub(r'([A-Z])',r'_\1', c).lower() for c in rdw_closed.columns]

# Basic operations
rdw_closed['ovi_private_owners'] = rdw_closed.ovi_eigenaren.str.split('/').apply(lambda x:int(x[0].strip()))
rdw_closed['ovi_company_owner'] = rdw_closed.ovi_eigenaren.str.split('/').apply(lambda x:int(x[1].strip()))
rdw_closed['ovi_owners'] = rdw_closed['ovi_private_owners'] + rdw_closed['ovi_company_owner']
rdw_closed['ovi_under_survey'] = rdw_closed.ovi_wachten_op_keuring.apply(lambda x: {'Ja': True, 'Nee': False}[x])

df = df.merge(rdw_closed, how='left', left_on='kenteken', right_index=True)

### Merge rdw and drz

In [17]:
# merge auction results and rdw queries. Add prefix "rdw_"
rdw = pd.concat([drz, df.add_prefix('rdw_')], axis='columns', sort=False)
# There should be no duplicates in column names
assert all(rdw.columns.value_counts() == 1)
# indices should match
assert rdw.index.isin(drz.index).all() & drz.index.isin(rdw.index).all()

out = rdw.copy()

# Saving

In [ ]:
if OPBOD:
    file_name = f'../../../python-nb/data/rdw-data-opbod-{DATE}.pkl'
else:
    file_name = f'../data/rdw-data-{DATE}.pkl'

if NO_PRICE:
    file_name = file_name.replace('.pkl', '-without-price.pkl')
    
if (SKIPSAVE==False) and (not(os.path.isfile(file_name))):
    print(file_name)
    out.to_pickle(file_name)
else:
    print(f'Skip. {file_name} exists or saving is disabled in settings.')

# Next: download images (or parallel)

Because images might be taken down from the drz site, it is advisable to run the notebook that downloads images soon.

In [206]:
assert False, 'Stop running. Below is to check if existing file has same dataframe.'

AssertionError: Stop running. check if existing file has same dataframe

In [ ]:
rdw = pd.read_pickle('../data/rdw-data-2021-06-opzij.pkl')

In [ ]:
is_eq = rdw.eq(out)
is_na = rdw.isna() & out.isna()

In [ ]:
col = 'Raw_text'

c = 0
for aa,bb in zip(rdw[col],out[col]):
    c+=1
    sd1 = np.setdiff1d(aa,bb)
    sd2 = np.setdiff1d(bb,aa)
    if (len(sd1) == 1 & len(sd2) == 1) & (sd2[0][:-1] == sd1[0]):
        continue
    else:
        if sd1:
            for l in sd1:
                print(f'|{l}|')
        if sd2:
            for l in sd2:
                print(f'|{l}|')
        raise
            
is_eq[col] = True

In [ ]:
col = 'LotCat'

c = 0
for aa,bb in zip(rdw[col],out[col]):
    c+=1
    sd1 = np.setdiff1d(aa,bb)
    sd2 = np.setdiff1d(bb,aa)
    if (len(sd1) == 1 & len(sd2) == 1) & (sd2[0][:-1] == sd1[0]):
        continue
    else:
        if sd1:
            for l in sd1:
                print(f'|{l}|')
        if sd2:
            for l in sd2:
                print(f'|{l}|')
        raise
            
is_eq[col] = True

In [ ]:
sel = (is_na | is_eq)

sel.loc[:, sel.columns.str.startswith('rdw_TimeStamp')] = True


In [ ]:
pd.concat([
    pd.concat([rdw.loc[(sel == False).any(axis=1), sel.all() == False]], keys=['on disk'], axis=1),
    pd.concat([out.loc[(sel == False).any(axis=1), sel.all() == False]], keys=['memory'], axis=1)
], axis = 1).sort_index(level=1, axis=1)


In [ ]:
for col in rdw.loc[:, sel.all() == False].columns:
    print(col)
    display(pd.concat([
        rdw.loc[sel[col]==False, col], 
        out.loc[sel[col]==False, col]
    ], axis=1))